In [1]:
import numpy as np
import helper_functions as hf
from scipy.stats import kurtosis, skew
from biosppy.signals.emg import emg

In [2]:
X_train_eeg1, _ = hf.read_csv_to_matrix("task5/input/train_eeg1.csv", "Id")
X_train_eeg2, _ = hf.read_csv_to_matrix("task5/input/train_eeg2.csv", "Id")
X_train_emg, _ = hf.read_csv_to_matrix("task5/input/train_emg.csv", "Id")
X_test_eeg1, _ = hf.read_csv_to_matrix("task5/input/train_eeg1.csv", "Id")
X_test_eeg2, _ = hf.read_csv_to_matrix("task5/input/train_eeg2.csv", "Id")
X_test_emg, _ = hf.read_csv_to_matrix("task5/input/train_emg.csv", "Id")
y_train, _ = hf.read_csv_to_matrix("task5/input/train_labels.csv", "Id")
_, test_index = hf.read_csv_to_matrix("task5/input/sample.csv", "Id")
y_train = np.squeeze(y_train)

In [3]:
def eeg_feature_extraction(x):
    x_new = []
    for i in range(x.shape[0]):
        x_new.append([np.mean(x[i]), np.std(x[i]), kurtosis(x[i]), skew(x[i])])
    return np.asarray(x_new)

In [4]:
X_train_eeg1_new = eeg_feature_extraction(X_train_eeg1)

In [27]:
X_test_emg_new = []
ts = []
filtered = []
onsets = []
for i in range(X_train_emg.shape[0]):
    ts[i], filtered[i], onsets[i] = emg(X_train_emg[i])

In [32]:
emg(X_train_emg[0])

NameError: name 'emg' is not defined